In [20]:
import numpy as np

import sys
print(sys.getrecursionlimit())
sys.setrecursionlimit(5000)

3000


In [21]:
class Day16():

    def __init__(self, data_link):
        data_txt = open(data_link, 'r').read()
        self.board = np.array([[letter for letter in elem] for elem in data_txt.split('\n')[:-1]])
        self.pos = np.argwhere(self.board == 'S')[0]
        self.end_pos = np.argwhere(self.board == 'E')[0]
        self.good_paths = []

    def display(self):
        print('\n'.join([''.join(list(line)) for line in self.board]))
        print()

    def get_neigh(self, pos = None):
        if not pos:
            pos = self.pos
        symbol = {(1, 0):'v', (-1, 0):'^', (0, 1):'>', (0, -1):'<'}
        return [(pos[0]+dir[0], pos[1]+dir[1], symbol[dir]) for dir in [(1, 0), (-1, 0), (0, 1), (0, -1)] if self.board[pos[0]+dir[0], pos[1]+dir[1]] in ['.', 'E']]

    def simplify(self, pos):
        neighs_pt = [(pos[0]+dir[0], pos[1]+dir[1]) for dir in [(1, 0), (-1, 0), (0, 1), (0, -1)] if self.board[pos[0]+dir[0], pos[1]+dir[1]] == '.']
        if len(neighs_pt) == 1:
            self.board[pos[0], pos[1]] = '#'
            self.simplify(neighs_pt[0])
        if len(neighs_pt) == 0:
            self.board[pos[0], pos[1]] = '#'
        return self

    def simplify_board(self):
        dead_ends = []
        for pos in np.argwhere(self.board == '.'):
            neighs_pt = [(pos[0]+dir[0], pos[1]+dir[1]) for dir in [(1, 0), (-1, 0), (0, 1), (0, -1)] if self.board[pos[0]+dir[0], pos[1]+dir[1]] in ['.', 'E', 'S']]
            if len(neighs_pt) <= 1:
                dead_ends.append(pos)
        for end in dead_ends:
            self.simplify(end)
        return self

    def solve(self, path = ['>']):
        neighs = self.get_neigh()
        if not neighs:
            return False
        for neigh in neighs:
            old_path = path.copy()
            path.append(neigh[2])
            if self.board[neigh[0], neigh[1]] == 'E':
                self.good_paths.append(path)
            else:
                old_pos, self.pos = tuple(self.pos), (neigh[0], neigh[1])
                self.board[old_pos] = 'o'
                self.solve(path)
                self.board[old_pos] = '.'
                self.pos = old_pos
            path = old_path.copy()

        return True

    def score(self, path):
        return len([path[i] for i in range(1, len(path)-1) if path[i-1]!=path[i]]) * 1000 + len(path)-1

    def all_score(self):
        return [self.score(path) for path in self.good_paths]

    def best_score(self):
        return min(self.all_score())


In [22]:
day16_test = Day16('other/day16_test.txt')
day16_test.display()
day16_test.simplify_board()
day16_test.display()
day16_test.solve()
day16_test.best_score()

###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############

###############
#...#####....E#
#.#.#####.###.#
#.....###...#.#
#.###.#####.#.#
#.###.......#.#
#.#######.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S###.....#...#
###############



7036

: 

In [ ]:
day16 = Day16('data/day16.txt')
day16.display()
day16.simplify_board()
day16.display()
day16.solve()
day16.best_score()

#############################################################################################################################################
#.....#...#.....#...#.........#...#.....#...........#.........#...................#.#.........#.......#.........#.......#.......#.........#E#
#.#.#.#.###.#.#.#.#.#.###.###.###.#.#.#.###.#######.#.#.#####.#.#########.#####.#.#.#.#.###.#.#####.#.###.#####.#.#####.#.###.#.#.#######.#.#
#.#.#.............#...#...#.#...#.#.#.....................#...#.#.......#.#.....#.#.....#...#.......#...#.....#...#...#...#.#...#.....#.#.#.#
#.#.#.#####.#.#.#.#####.###.###.#.#.#.#.#.###.#.#.#.#####.###.###.#####.###.#####.#####.#.#############.#.###.#.###.#.#####.#.###.###.#.#.#.#
#.#.#.....#.#.#.#.#.#...#.....#.#...#.#.#.#.#...#...#...#...#...#.....#.....#...#.....#.#...#...#.....#.#...#.#.....#.........#.#...#.#.#.#.#
###.#####.#.#.###.#.#.###.###.#.#.###.#.#.#.#.#######.#.###.###.#.###.#########.#####.#.###.#.###.###.#.#.###.#.###############.###.#.#.#.#.#
#...#.